In [ ]:
from source.config import DATA_DIR, RAW_DATA_DIR, PROCESSED_DATA_DIR, REPORTS_DIR
from source.utils import sanitize_filename
from source.features_dir.estimated_registrations import SUBPATH
from pathlib import Path
import pandas as pd
from source.report import generate_latex_with_multiple_tables
import os

In [ ]:
def percentage_74t_registrations(subpath: str):
    
    def get_all_registrations(unsanitized_location: str):
        '''
        
        '''
        sanitized_filename = sanitize_filename(unsanitized_location)
        df = pd.read_csv(RAW_DATA_DIR / 'estimated_registrations' / 'total_registrations_bwim74t' / f'{sanitized_filename}.csv', sep=';', encoding_errors='ignore')
        df = df[df['Felt'] == 'Totalt']
        df['Year'] = pd.to_datetime(df['Fra']).dt.year
        return df

    processed_data_project_path = PROCESSED_DATA_DIR / 'estimated_registrations' / subpath
    latex_project_path = REPORTS_DIR / 'estimated_registrations' / subpath

    df_truck = pd.read_csv(processed_data_project_path / 'final-truck_only.csv')
    df_trailer = pd.read_csv(processed_data_project_path / 'final-trailer_only.csv')
    dfs = [df_truck, df_trailer]


    for i, df_74t in enumerate(dfs):
        final_info = []
        for road in df_74t['Vei'].to_list():
            data = []   
            df_all_registrations_for_road = get_all_registrations(road)
            for year in [2021, 2022, 2023, 2024]:
                all_registrations_for_year_in_16_24_category = df_all_registrations_for_road[df_all_registrations_for_road['Year'] == year]['16,0m - 24,0m']
                all_registrations_for_year_in_above_24_category = df_all_registrations_for_road[df_all_registrations_for_road['Year'] == year]['>= 24,0m']
                all_registrations_year_16_and_above = all_registrations_for_year_in_16_24_category + all_registrations_for_year_in_above_24_category
                
                n_74t_registrations_year_road_60 = df_74t[df_74t['Vei'] == road][f'{year} 60t']
                n_74t_registrations_year_road_65 = df_74t[df_74t['Vei'] == road][f'{year} 65t']
                n_74t_registrations_year_road_68 = df_74t[df_74t['Vei'] == road][f'{year} 68t']
                n_74t_registrations_year_road_74 = df_74t[df_74t['Vei'] == road][f'{year} 74t']
                n_74t_registrations_year_road_total = n_74t_registrations_year_road_60 + n_74t_registrations_year_road_65 + n_74t_registrations_year_road_68 + n_74t_registrations_year_road_74

                percentage_74t_registrations_year_road = n_74t_registrations_year_road_total / all_registrations_year_16_and_above * 100

                data.append({
                    'År': year,
                    '60t (3+4)': n_74t_registrations_year_road_60,
                    '65t (4+4)': n_74t_registrations_year_road_65,
                    '68t (3+5)': n_74t_registrations_year_road_68,
                    '74t (4+5)': n_74t_registrations_year_road_74,
                    'Totalt BK74': n_74t_registrations_year_road_total,
                    'Prosent BK74': percentage_74t_registrations_year_road
                })

            df = pd.DataFrame(data)
            os.makedirs(processed_data_project_path /'percentages', exist_ok=True)
            df.to_csv(processed_data_project_path /'percentages' / f'{'trailer' if i else 'truck'}_percentages{road}.csv', index=False)
            caption = f'Prosentandel BK74 {'tilhenger' if i else 'lastebil'}-registreringer av alle trafikkdata-registreringer for {road}'
            final_info.append((df, caption))

    generate_latex_with_multiple_tables(
        output_dir=REPORTS_DIR / 'estimated_registrations' / subpath,
        filename=f'percentage_74t_registrations',
        title=f'Prosentandel BK74 registreringer av alle trafikkdata-registreringer',
        tables=final_info
    )

percentage_74t_registrations(SUBPATH)